In [22]:
import pymongo
import json
import requests
import time

In [23]:
import pandas as pd
data = pd.read_csv(r"C:\Users\frank\OneDrive\Desktop\FoodDoooom\Scraper\DB_data\GroceryDB.CleanedData.csv").head()
data_dict = data.set_index('original_ID').T.to_dict('dict')
print(data_dict)

{'tg_11213449': {'Protein Conv': nan, 'Total Fat Conv': nan, 'Carbohydrate Conv': nan, 'Sugars, total Conv': nan, 'Fiber, total dietary Conv': nan, 'Calcium Conv': nan, 'Iron Conv': nan, 'Sodium Conv': nan, 'Vitamin C Conv': nan, 'Cholesterol Conv': nan, 'Total Vitamin A Conv': nan, 'calories Conv': nan}, 'tg_11215316': {'Protein Conv': '[0.0, "g", 0.0]', 'Total Fat Conv': '[0.0, "g", 0.0]', 'Carbohydrate Conv': '[29.0, "g", 29.0]', 'Sugars, total Conv': '[27.0, "g", 27.0]', 'Fiber, total dietary Conv': nan, 'Calcium Conv': '[30.0, "mg", 0.03]', 'Iron Conv': nan, 'Sodium Conv': '[105.0, "mg", 0.105]', 'Vitamin C Conv': nan, 'Cholesterol Conv': nan, 'Total Vitamin A Conv': nan, 'calories Conv': '[110.0, null, 110.0]'}, 'tg_11215317': {'Protein Conv': nan, 'Total Fat Conv': '[0.0, "g", 0.0]', 'Carbohydrate Conv': '[2.0, "g", 2.0]', 'Sugars, total Conv': nan, 'Fiber, total dietary Conv': nan, 'Calcium Conv': nan, 'Iron Conv': nan, 'Sodium Conv': '[105.0, "mg", 0.105]', 'Vitamin C Conv': n

In [24]:
def count_response_codes(urls):
    
    """This function will give us the statistic on how many products are currently offered in the respective store."""
    
    status_counts = {"200": 0, "4xx": 0, "5xx": 0, "other": 0, "not_available_products": [] }

    for url in urls:
        try:
            print(url)
            response = requests.get(url, timeout=5)  # Timeout to prevent long waits
            status_code = response.status_code

            if status_code == 200:
                status_counts["200"] += 1
            elif 400 <= status_code < 500:
                status_counts["4xx"] += 1 
                status_counts["not_available_products"].append(url)
            elif 500 <= status_code < 600:
                status_counts["5xx"] += 1 
            else:
                status_counts["other"] += 1 

        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            status_counts["other"] += 1  # Count failed requests

    return status_counts

count_response_codes(data["original_ID"])


0    tg_11213449
1    tg_11215316
2    tg_11215317
3    tg_11853100
4    tg_12190462
Name: original_ID, dtype: object
tg_11213449
Error fetching tg_11213449: Invalid URL 'tg_11213449': No scheme supplied. Perhaps you meant https://tg_11213449?
tg_11215316
Error fetching tg_11215316: Invalid URL 'tg_11215316': No scheme supplied. Perhaps you meant https://tg_11215316?
tg_11215317
Error fetching tg_11215317: Invalid URL 'tg_11215317': No scheme supplied. Perhaps you meant https://tg_11215317?
tg_11853100
Error fetching tg_11853100: Invalid URL 'tg_11853100': No scheme supplied. Perhaps you meant https://tg_11853100?
tg_12190462
Error fetching tg_12190462: Invalid URL 'tg_12190462': No scheme supplied. Perhaps you meant https://tg_12190462?


{'200': 0, '4xx': 0, '5xx': 0, 'other': 5, 'not_available_products': []}

In [55]:
def build_url(product_id):
    """Construct the API URL using the product ID."""
    base_url = "https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1?key=9f36aeafbe60771e321a7cc95a78140772ab3e96&tcin={}&is_bot=false&pricing_store_id=3324&visitor_id=019347C3E9160201A06B297D0CC373F6"
    return base_url.format(product_id)

def fetch_nutritional_data(url, headers):
    """Make a request to the API and fetch nutritional data."""
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed for URL {url} with status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error while making the request: {e}")
        return None

def extract_nutritional_values(data):
    
    """Extract and format nutritional values from the API response."""
    nutritional_data = {}
    nutrition_facts = data.get('data', {}).get('product', {}).get('item', {}).get('enrichment',{}).get('nutrition_facts',{})
    value_prepared_list = nutrition_facts.get('value_prepared_list', [])
    
    if value_prepared_list:
        nutrients = value_prepared_list[0].get('nutrients', [])
        for nutrient in nutrients:
            name = nutrient.get('name')
            quantity = nutrient.get('quantity')
            unit = nutrient.get('unit_of_measurement')
            if name and quantity is not None:
                nutritional_data[name] = {"quantity": quantity, "unit_of_measurement": unit}
    return nutritional_data


In [92]:
flaggedRecords = []
error_count = 0

def checkForDifference(nutritional_values_current):
    
    DEVIATION_TOLERANCE = 1
    global foodRecords
    
    for productID in nutritional_values_current.keys():
        #try:
            nutritional_values_database = data_dict.get("tg_"+productID, None)
            if not nutritional_values_database:
                flaggedRecords.append(f"tg_{productID}: Database record not found")
                continue
            
            print("checked")

            nutrients_to_check = {
                "Total Fat": "Total Fat Conv",
                "Cholesterol": "Cholesterol Conv",
                "Sodium": "Sodium Conv",
                "Total Carbohydrate": "Carbohydrate Conv",
                "Dietary Fiber": "Fiber, total dietary Conv",
                "Total Sugars": "Sugars, total Conv",
                "Protein": "Protein Conv",
                "Calcium": "Calcium Conv",
                "Iron": "Iron Conv",
                "Calories": "calories Conv",
                "Vitamin A": "Total Vitamin A Conv",
                "Vitamin C": "Vitamin C Conv"
            }

            for nutrient, db_field in nutrients_to_check.items():
                if nutrient in nutritional_values_current[productID]:
                    try:
                        db_value = nutritional_values_database[db_field]
                        if isinstance(db_value, str):
                            db_value = json.loads(db_value)
                        if isinstance(db_value, list):
                            db_value = float(db_value[0])
                        else:
                            db_value = float(db_value)
                        current_value = float(nutritional_values_current[productID][nutrient]["quantity"])

                        if abs(db_value - current_value) > DEVIATION_TOLERANCE:
                            flaggedRecords.append(f"tg_{productID}: {nutrient} deviation (DB: {db_value}, Current: {current_value})")
                            break
                    except (KeyError, ValueError, TypeError) as e:
                        flaggedRecords.append(f"tg_{productID}: Error processing {nutrient} - {e}")
                        break
        

    print(flaggedRecords)
    
    

def fetch_nutritional_values():
    
    """Main function to fetch and store nutritional values."""
    
    data = pd.read_csv(r"C:\Users\frank\OneDrive\Desktop\FoodDoooom\Scraper\DB_data\GroceryDB.CleanedData.csv")
    print("Hello")
    data_dict = data.set_index('original_ID').T.to_dict('dict')

    original_ids = data["original_ID"][0:100]
    print(original_ids)
    
    headers = {
        "User-Agent": "PostmanRuntime/7.43.0",
        "Accept": "*/*",
        "Cache-Control": "no-cache",
        "Postman-Token": "37892edf-2bc5-4d16-9936-dc39def9bb66",
        "Host": "redsky.target.com",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Cookie": "TealeafAkaSid=Di5RRXbgfIeZiq0-c3T4YQpuPsxcgwHW; sapphire=1; visitorId=0194BE4F07B90201BBFC5093D03D4773; _tgt_session=fe687524979441b7b1f28b3e5bbd2240.eb4775f1fbcf975298b2e3e4c7753028ab706e05e4adf6b45f0ee124cb5542a4889648d075a393b34b4971c1f68005f8f59b8c13bac99bd4e21435405da4ebf4a7939477e6a7e36a06cd61db46984f93bc984561680f6d4be8bfde3fb44969e4aac8a388ad010d01a60c0f6e97c257cc47d6ed8d23067487ceb6d30f7f798c2da63e795a73094c99c52ecf27a77b7f7013184d6f91df851fbe436f6a8ce6a71c21395579465b9e3b186883ac37c8729ab7f6c33a1f5aa688e7c954045153e163bbf944ce7ea3e284401a818530998856a6f834cfda3526782fa5d856a51e9e8175.0xb0d9bcc05c5c68728aad78687699da4b1f2d90ac3b68f4f628f4dc838e0ceac4"
    }
    nutritional_values = {}
    global foodRecords
    
    for product_id in original_ids:
        try:
            if not product_id.startswith("tg"):
                continue
            new_id = product_id[3:]
            if new_id.isdigit():
                url = build_url(new_id)
                print(f"Making request for URL: {url}")
                
                data = fetch_nutritional_data(url, headers)
                if data:
                    nutritional_values[new_id] = extract_nutritional_values(data)
                time.sleep(3)
        except Exception as e:
            if "Error" in str(e):
                error_count += 1
                if error_count >= 100:
                    print("Error count exceeded 100. Stopping further processing.")
                    break
            flaggedRecords.append(f"Error processing product ID {product_id}: {e}")
            
    checkForDifference(nutritional_values)
    return nutritional_values

a = fetch_nutritional_values()

Hello
0     tg_11213449
1     tg_11215316
2     tg_11215317
3     tg_11853100
4     tg_12190462
         ...     
95    tg_12935436
96    tg_12935440
97    tg_12935441
98    tg_12935442
99    tg_12935452
Name: original_ID, Length: 100, dtype: object
Making request for URL: https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1?key=9f36aeafbe60771e321a7cc95a78140772ab3e96&tcin=11213449&is_bot=false&pricing_store_id=3324&visitor_id=019347C3E9160201A06B297D0CC373F6
Making request for URL: https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1?key=9f36aeafbe60771e321a7cc95a78140772ab3e96&tcin=11215316&is_bot=false&pricing_store_id=3324&visitor_id=019347C3E9160201A06B297D0CC373F6
Making request for URL: https://redsky.target.com/redsky_aggregations/v1/web/pdp_client_v1?key=9f36aeafbe60771e321a7cc95a78140772ab3e96&tcin=11215317&is_bot=false&pricing_store_id=3324&visitor_id=019347C3E9160201A06B297D0CC373F6
Making request for URL: https://redsky.target.com/redsky_aggreg

In [93]:
for record in flaggedRecords:
    print(f"Flagged Record: {record}")


Flagged Record: tg_12190967: Database record not found
Flagged Record: tg_12918876: Database record not found
Flagged Record: tg_12918877: Database record not found
Flagged Record: tg_12918878: Database record not found
Flagged Record: tg_12920261: Database record not found
Flagged Record: tg_12920692: Database record not found
Flagged Record: tg_12920741: Database record not found
Flagged Record: tg_12920863: Database record not found
Flagged Record: tg_12921029: Database record not found
Flagged Record: tg_12921030: Database record not found
Flagged Record: tg_12921061: Database record not found
Flagged Record: tg_12921162: Database record not found
Flagged Record: tg_12921317: Database record not found
Flagged Record: tg_12921319: Database record not found
Flagged Record: tg_12921514: Database record not found
Flagged Record: tg_12921519: Database record not found
Flagged Record: tg_12921525: Database record not found
Flagged Record: tg_12921529: Database record not found
Flagged Re